# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [32]:
import os
from glob import glob

price_data_dir = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(price_data_dir, "**", "*.parquet"), recursive=True)
normalized_paths = [path.replace("\\", "/") for path in parquet_files]


for file_path in normalized_paths:
    print(file_path)

../../05_src/data/prices/ACN/ACN_2001/part.0.parquet
../../05_src/data/prices/ACN/ACN_2001/part.1.parquet
../../05_src/data/prices/ACN/ACN_2002/part.0.parquet
../../05_src/data/prices/ACN/ACN_2002/part.1.parquet
../../05_src/data/prices/ACN/ACN_2003/part.0.parquet
../../05_src/data/prices/ACN/ACN_2003/part.1.parquet
../../05_src/data/prices/ACN/ACN_2004/part.0.parquet
../../05_src/data/prices/ACN/ACN_2004/part.1.parquet
../../05_src/data/prices/ACN/ACN_2005/part.0.parquet
../../05_src/data/prices/ACN/ACN_2005/part.1.parquet
../../05_src/data/prices/ACN/ACN_2006/part.0.parquet
../../05_src/data/prices/ACN/ACN_2006/part.1.parquet
../../05_src/data/prices/ACN/ACN_2007/part.0.parquet
../../05_src/data/prices/ACN/ACN_2007/part.1.parquet
../../05_src/data/prices/ACN/ACN_2008/part.0.parquet
../../05_src/data/prices/ACN/ACN_2008/part.1.parquet
../../05_src/data/prices/ACN/ACN_2009/part.0.parquet
../../05_src/data/prices/ACN/ACN_2009/part.1.parquet
../../05_src/data/prices/ACN/ACN_2010/part.0.p

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

def add_features(x):
    return x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        returns = x['Close'] / x['Close'].shift(1) - 1,
        hi_lo_range = x['High'] - x['Low']
    )

dd_feat = dd_px.groupby('ticker', group_keys=False).apply(add_features)

C:\Users\annam\AppData\Local\Temp\ipykernel_13864\4139134364.py:11: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(add_features)#.reset_index()


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
df = dd_feat.compute().reset_index()
df["returns_ma_10"] = (
    df.sort_values("Date")
       .groupby("ticker")["returns"]
       .transform(lambda x: x.rolling(window=10).mean()))
df.tail()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
362767,ZIXI,2020-03-26,4.06,4.53,3.88,4.51,4.51,1668500.0,ZIXI.csv,2020,4.00,4.00,0.127500,0.65,-0.000807
362768,ZIXI,2020-03-27,4.49,4.71,4.10,4.60,4.60,1146800.0,ZIXI.csv,2020,4.51,4.51,0.019956,0.61,0.004262
362769,ZIXI,2020-03-30,4.83,4.87,4.44,4.64,4.64,1212000.0,ZIXI.csv,2020,4.60,4.60,0.008696,0.43,0.015491
362770,ZIXI,2020-03-31,4.60,4.69,4.10,4.31,4.31,1057200.0,ZIXI.csv,2020,4.64,4.64,-0.071121,0.59,0.015926
362771,ZIXI,2020-04-01,4.11,4.16,3.80,3.82,3.82,539500.0,ZIXI.csv,2020,4.31,4.31,-0.113689,0.36,0.027261


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It depends on the use case. If data is small to medium-size, converting to Pandas is fine and as it gives access to rolling(), .groupby(), and other rich features. But with large/distributed data, converting to Pandas can blow up memory usage, slow down the pipeline, result in a loss of parallelism and scalability.  Advantages of staying in Dask are parallelism, memory efficiencyp, ipeline consistency.  But there are some challenges.  Dask’s API is more limited—especially for .rolling() and .groupby().apply() across partitions.



## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.